In [1]:
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from docx.shared import RGBColor

# Membaca data dari CSV
data = pd.read_csv('rill_final_enhanced/data/cv_detailed_results_all_scenarios.csv')

# Menghitung rata-rata dan standar deviasi untuk setiap model dan skenario
metrics = ['ROC_AUC', 'Accuracy', 'Precision', 'Recall', 'F1']
grouped = data.groupby(['Scenario', 'Model'])[metrics].agg(['mean', 'std']).reset_index()

# Membuat dokumen Word
doc = Document()
doc.add_heading('Tabel 1. Hasil 5-Fold Cross-Validation untuk Berbagai Model dan Skenario', level=1)

# Membuat tabel
table = doc.add_table(rows=1, cols=6)
table.style = 'Table Grid'
table.autofit = True

# Menambahkan header tabel
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Skenario / Model'
hdr_cells[1].text = 'ROC-AUC'
hdr_cells[2].text = 'Accuracy'
hdr_cells[3].text = 'Precision'
hdr_cells[4].text = 'Recall'
hdr_cells[5].text = 'F1-Score'

# Mengatur font header
for cell in hdr_cells:
    cell.paragraphs[0].runs[0].font.bold = True
    cell.paragraphs[0].runs[0].font.name = 'Times New Roman'
    cell.paragraphs[0].runs[0].font.size = Pt(11)

# Mengelompokkan data berdasarkan skenario
scenarios = ['No_FE', 'SMOTE_Only','FE_Only', 'FE_SMOTE']
for scenario in scenarios:
    # Menambahkan baris untuk nama skenario (subjudul)
    row = table.add_row().cells
    row[0].text = scenario
    row[0].paragraphs[0].runs[0].font.bold = True
    row[0].paragraphs[0].runs[0].font.name = 'Times New Roman'
    row[0].paragraphs[0].runs[0].font.size = Pt(11)
    for i in range(1, 6):
        row[i].text = ''

    # Filter data untuk skenario saat ini
    scenario_data = grouped[grouped['Scenario'] == scenario]

    # Menemukan nilai maksimum untuk setiap metrik
    max_values = scenario_data[metrics].max()

    # Menambahkan baris untuk setiap model
    for _, row_data in scenario_data.iterrows():
        model = row_data['Model']
        row = table.add_row().cells
        row[0].text = f'  {model}'  # Indentasi untuk model
        row[0].paragraphs[0].runs[0].font.name = 'Times New Roman'
        row[0].paragraphs[0].runs[0].font.size = Pt(11)

        # Mengisi metrik
        for i, metric in enumerate(metrics, 1):
            mean = row_data[(metric, 'mean')]
            std = row_data[(metric, 'std')]
            cell_text = f'{mean:.3f}±{std:.3f}'
            row[i].text = cell_text
            run = row[i].paragraphs[0].runs[0]
            run.font.name = 'Times New Roman'
            run.font.size = Pt(11)

            # Tandai nilai terbaik dengan teks tebal
            if mean == max_values[metric]['mean']:
                run.font.bold = True
                run.font.color.rgb = RGBColor(0, 0, 0)

# Menambahkan catatan kaki
doc.add_paragraph(
    'Catatan: Nilai yang ditampilkan adalah rata-rata ± standar deviasi dari 5-fold cross-validation. '
    'No_FE: Tanpa Feature Engineering; FE_Only: Dengan Feature Engineering; '
    'FE_SMOTE: Feature Engineering dengan SMOTE. Nilai terbaik untuk setiap metrik ditandai dengan tebal.'
).style = 'Caption'

# Mengatur font catatan kaki
for run in doc.paragraphs[-1].runs:
    run.font.name = 'Times New Roman'
    run.font.size = Pt(10)
    run.font.italic = True

# Menyimpan dokumen
doc.save('cross_validation_results.docx')